# Unidad 2 - 01 Regresión Lineal (Ridge y Lasso) en procesos de ML

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Introducción

Los modelos que surgen de las estadísticas suelen ser simples, pero la inherente aleatoriedad de los datos hace que haya que considerar sutilezas cuando se trata de adaptar el modelo a los datos. En general este proceso de ajuste tiene que ver con la minimización del error, es decir, las diferencias entre las prediciones del modelo y el dato real *(residuales)*.

La tarea de minimizar el error es delicada, podemos caer facilmente en una situación de overfitting (que desarrollaremos brevemente más adelante). Distintas elecciones para la norma a minimizar conducen a distintos modelos que se adaptan mejor según las condiciones de los datos. Por lo tanto el proceso de selección de la misma no es tarea sencilla. Como matemáticos se espera que podamos entender mejor las fórmulas y sutilezas que resultan en los distintos modelos, decidir con mejor criterio si un modelo adapta bien (o no) a los datos y en consecuencia, hacer mejores predicciones.

La regresión lineal es el modelo más común de la estadística. Se utiliza para modelizar una relación lineal entre 2 variables cuantitativas.




## 2. Regresión Lineal. Minimización del error residual.

Cuando adaptamos un modelo lineal, en general contamos con un archivo de datos (en formato CSV probablemente), como es el caso de la tabla siguiente:

![GaltonDataset.PNG](https://drive.google.com/uc?export=view&id=1AOoPZCYfrS48Sg_9qV4N2hRPlSx2DHou)


Un modelo lineal  establece que $y=\beta_0+\beta_1x+\varepsilon$, donde $\varepsilon$ es un término de error aleatorio, cuyo valor esperado es $\textbf{cero}$.

Para el ejemplo del conjunto de datos de la tabla, (Galton dataset) un software de estadística puede computar "el" modelo lineal "mas ajustado" y declarar que el mismo viene dado por: $y=39.11+0.4x+\varepsilon$ (1).

![LRforGaltonDS.PNG](https://drive.google.com/uc?export=view&id=15mKdhmaNarRIPSQKZMxeYA85QCssoEQc)

Dicho modelo, además de explicar la relación entre los datos (aparentemente lineal), puede ser utilizado para $\textbf{predecir}$ valores de $y$ asociados a valores de $x$ dados.

Usamos la notación $\hat{y}$ para un valor predicho de $y$, obtenido de sustituir un valor de $x$ en la ecuación de regresión (1). Para los valores $x$ de la tabla podemos calcular los residuales.

Si llamamos $\beta_1$ a la pendiente real en la población (Familias de Gran Bretaña de los 80's) y $\hat{\beta_1}$ pendiente de la  muestra (muestra aleatoria específica de 898 familias) y similar para $\beta_0$ y $\hat{\beta_0}$, entonces, el ajuste del modelo lineal se reduce a determinar $\hat{\beta_1}$ y $\hat{\beta_0}$ de manera tal que minimizen $\textbf{alguna medida del error residual}$.

Dicha "medida" puede ser:

- La suma de los cuadrados de los residuos (RSS)
- la varianza (regularización $l_2$, regresión Ridge)
- la varianza (regularización $l_1$, regresión LASSO)

Cada uno de ellos se encara como un problema de optimización con distintas características, y por lo tanto se utilizan diferentes herramientas (del área de optimización) para encontrar la solución.

- Mínimos cuadrados ordinarios
- Optimización con restricciones: multiplicadores de Lagrange
- Descenso del gradiente (no hay solución explícita, ni resultados de unicidad, se recurre a la optimización numérica).

Bajo las hipótesis clásicas de de regresión:
- La relación entre $x$ e $y$ es lineal
- Los errores $\varepsilon$ tienen media $\textbf{cero}$
- Los errores son homoscedásticos (i.e. todos los errores residuales tienen la misma varianza)
- los datos $(x)$ son independientes
- en el caso de que existan múltiples variables $x$, deben ser linealmente independientes,

el teorema de Gauss-Markov, establece que las fórmulas obtenidas vía mínimos cuadrados ordinarios para $\hat{\beta_1}$ y $\hat{\beta_0}$, nos dan los "mejores" estimadores no cesgados para el modelo (i.e. en el sentido que hacen mínima la varianza). Sin embargo, se necesitan alternativas para la medida del error a minimizar, ya que las hipótesis clásicas de regresión no siempre se verifican. Veremos qué opciones son apropiadas en distintas situaciones.



## El modelo lineal general

En su versión mas general, el modelo lineal establece que $Y=XB+\varepsilon$, donde:
- $Y \in \mathbb M_{n\times p}$, matriz de variables de respuesta ($p$), nos quedaremos con el caso sencillo de $p=1$,
- $X\in \mathbb M_{n\times(k+1)}$ matriz de las variables explicativas (y una columna extra para $\beta_0$),
- $B\in\mathbb M_{(k+1)\times 1}$ matriz de los $\beta_i$ que deben ser estimados y
- $\varepsilon\in\mathbb M_{n\times 1}$ matriz de errores.

### 2.1 Mínimos cuadrados ordinarios

Luego de decidir que el modelo lineal es apropiado, la tarea siguiente es ajustar el modelo a los datos. Esto implica encontrar estimaciones de $\hat{\beta}$ para los $\beta$-coeficientes, lo cual puede hacerse de diversas maneras.

Primera propuesta:

Considerar la suma de los cuadrados de los residuos como función de $\beta_i (k+1)$ variables, calcular derivadas parciales, igualar a cero y optimizar. Las ecuaciones resultantes de este planteo  se denominan **ecuaciones normales**.

Segunda propuesta:

Proyectar $Y$ en el hiperplano $k$-dimensional generado por $x_i$. El vector proyección $\hat{Y}$ representa todo lo que las variables $X$ saben de $Y$. Con un enfoque de **álgebra lineal**, $\hat Y=PY$, con $P$ matriz de proyección.
Para hacer mínimos cuadrados ordinarios  debemos elegir $\hat B$ que minimice la suma de cuadrados de residuales, $||Y-XB||^2$, o equivalentemente, el producto punto $(Y-XB)^T(Y-XB)$. La distancia euclídea es minimizada por $\hat Y=X\hat B$, la proyección ortogonal.
Por la ortogonalidad precisamente, tenemos $X^T(Y-XB)=0$.

Las ecuaciones normales entonces, son equivalentes a la afirmación: $(X^TX)\hat B=X^TY$. Resulta, $\hat B=(X^TX)^{-1}X^TY$ siempre que $X^TX $ sea invertible. Aquí identificamos la necesidad de que las variables explicativas $(x)$ sean linealmente independientes.

Dado que $\hat Y=X\hat B$, sigue que $P=X(X^TX)^{-1}X^T$.

Las estimaciones así obtenidas para los $\beta_i$ coindicen con las del cálculo multivariable. El Teorema de Gauss-Markov establece que $\hat B$ es la mejor estimación lineal, no sesgada de $B$. La prueba requiere las hipótesis clásicas de regresión pero no necesita la normalidad multivariada de los errores.

TEMAS PARA DESARROLLAR: Teorema Gauss-Markov, soluciones de $\nabla = 0$, proyecciones ortogonales.

### 2.2 Regresión Ridge

 En la sección anterior concluímos que los estimadores $\hat \beta$ pueden obtenerse como solución a las ecuaciones normales $(X^TX)\hat B=X^TY$, pero cuando $X^TX$ no es invertible, el problema no tiene solución única y por lo tanto, algún criterio para elegir entre las soluciones, debe ser usado.

Primer criterio:

Elegir $B$ de mínima norma $||B||_2$ que minimiza $||XB-Y||_2^2$. La solución a este problema viene dada por $B=X^+Y$ donde $X^+$ es la pseudoinversa de $X$. Dicha $X^+$ es obtenida de un $SVD$ de $X$ (Desarrollaremos esta técnica más adelante).

La dificultad con este enfoque tiene que ver con valores singulares de $X$ cercanos a cero o cero, que puedan presentarse, y un fenómeno de discontinuidad no deseable que dicha situación presenta en el cálculo de $X^+$.

Segundo criterio:

Para controlar el tamaño de $B$, podemos agregar un término de regularización a la función a minimizar, digamos $K ||B||^2$. Esta propuesta se conoce como **regresión Ridge**.

El nuevo problema a resolver para encontrar estimadores $\hat B$ es entonces:
$$\underset{B}{\min} ||Y-XB||^2+K||B||^2,$$

que puede reescribirse como

$$\begin{array}{cc}\underset{\xi,B}{\min}&\xi^T\xi+K B^TB\\s.a.&Y-XB=\xi\end{array}$$

considerando la nueva variable $\xi=Y-XB$.

Con herramientas de optimización: gradientes, convexidad, problemas duales y multiplicadores de Lagrange para problemas con restricciones; se obtienen resultados para propuestas de estimadores $\hat B$.

En definitiva, el valor $K$ es un multiplicador de Lagrange que penaliza las estimaciones de $B$ que son muy grandes en la norma $l_2$. Claramente, la técnica usada en la regresión Ridge permite obtener unos estimadores $\hat B$ que reducen la varianza con el costo de introducir cierto sesgo en las estimaciones. Típico problema de Machine Learning. La Regresion Ridge es una herramienta poderosa en el caso de variables $X$ altamente correlacionadas.

DESARROLLAMOS...

### 2.3 Regresión Lasso

La principal debilidad de la regresión Ridge tiene que ver con el hecho de que usualmente, el vector $B$ estimado, tiene demasiados coeficientes NO nulos, en consecuencia, la regresión Ridge no va bien cuando el tamaño de los datos es grande. En la práctica debemos contar con métodos capaces de manejar millones de datos, o más. Una estrategia para fomentar la condición de "sparse" de nuestro vector $B$ es reemplazar el término de regularización de Ridge $K||B||^2_2$ por el término $K||B||_1$, en definitiva, cambiar  la norma $l_2$ por la norma $l_1$.

En este caso el problema de optimización a resolver para encontrar estimadores $\hat B$ es:
$$\begin{array}{cc}\underset{\xi,B}{min}& \dfrac{1}{2}\xi^T\xi+\tau||B||_1\\
s.a. & Y-XB=\xi\end{array}$$
donde $\tau$ es una constante que determina la influencia del término de regularización $||B||_1$.

Aquí las dificultades del término de regularización $||B||_1=|\beta_1|+|\beta_2|+...+|\beta_n|$ tienen que ver con el hecho de que la aplicación $B\rightarrow ||B||_1$ no es diferenciable para todo $B$. Dicha dificultad puede sortearse usando **subgradientes** o bien usando algunos trucos sencillos de cambio de variable, para luego obtener el problema dual a partir de la siguiente reformulación:

Siendo $|x|=max\{x,-x\}$ el problema de optimización se reescribe:

$$\begin{array}{cc}\underset{\xi,B,\epsilon}{min}& \dfrac{1}{2}\xi^T\xi+\tau \mathbb 1_n^Tϵ\\
s.a. & Y-XB=\xi\\
&B\leq ϵ\\
&-B\leq ϵ\end{array}$$

Las restricciones $B\leq ϵ$ y $-B\leq \epsilon $ son equivalentes a $|\beta_i|\leq\epsilon_i \forall i=1,..,n.$





##2.4 Comentarios

Tanto en caso Ridge como en el caso de la regularización Lasso hemos atendido el caso lineal con $\beta_0=0$, para mayor simplificación. El caso afín $Y=XB+B_0$, puede desarrollarse en cualquiera de las regularizaciones propuestas con las modificaciones apropiadas de los problemas de optimización a resolver.



## Ejemplo

Es interesante comparar el comportamiento de los distintos métodos mencionados para la elección de los parámetros: $B$ y $B_0$. Se comparan las rectas obtenidas sobre un conjunto de datos ($200$ pares $(X,Y)$) generados de manera aleatoria, para los siguientes métodos:
1. Regresión Ridge para el caso afín
2. Regresión Ridge con penalización del parámetro $B_0$
3. Mínimos cuadrados aplicados a $[X\; \mathbf 1]$
4. Regresión Lasso para el caso afín

1er caso:
$n=2,K=\tau=0.1$

No se observa diferencia notoria entre las rectas obtenidas por ser $K$ y $\tau$ pequeños y de la misma magnitud.

![LR1.PNG](https://drive.google.com/uc?export=view&id=1NGSw3ceCXdV04iFzhmCi4fXdr_zIZitR)


2do caso:
$n=2, K=1000, \tau=10000$

En este caso, la penalización de $B_0$ en el método 2, nos da una recta (en rojo) que se encuentra significativamente por debajo del resto. El valor más grande de $\tau$, hace que el método 4 nos dé una línea con menor pendiente (en cyan). El método 1 (en magenta) también tiene menor pendiente que el caso 1, pero sigue pareciéndose bastante a la línea que se obtiene con mínimos cuadrados (en azul).

![LR1.PNG](https://drive.google.com/uc?export=view&id=1W_nIb-xAQVviEGvm8H3J3vQXmmhx2I8x)




---


Estas notebooks corresponden a la asignatura electiva para la Licenciatura en Matemática del Departamento de Matemática de la Escuela de Cs. Exactas y Naturales, de la Facultad de Cs. Exactas, Ingenieria y Agrimensura (FCEIA) de la Universidad Nacional de Rosario, Argentina. Año 2023.

Fueron confeccionadas a tal fin por las docentes investigadoras de la FCEIA Isolda Cardoso y Jorgelina Walpen. Este trabajo de análisis, estudio, recopilación, traducción, armado, pruebas y errores, nos ha llevado mucho tiempo. Cuando hay tanta información disponible es complicado extraer lo que, al menos para nosotras, es relevante. Hay muchísimos otros recursos y tutoriales, nosotras armamos este.

Si estas notebooks son reproducidas, solicitamos citar la fuente.

Dejamos nuestras páginas laborales y repositorios de GitHub. Seguramente en un futuro las pondremos a disposición en el Repositorio Hipermedial de la UNR.

Isolda: Te dejo mi página laboral https://www.fceia.unr.edu.ar/~isolda/ y mi GitHub https://github.com/IsoldaEugenia. Sentite libre de contactarme.

Jorgelina: https://www.fceia.unr.edu.ar/~walpen/ y  https://github.com/JorWalpen